# RAG Коришћење Azure AI Agent Service и Semantic Kernel

Овај пример кода показује како да креирате и управљате Azure AI агентом за генерисање уз помоћ претраге (RAG) користећи `Azure AI Agent Service` и `Semantic Kernel`. Агент обрађује корисничке упите на основу пронађеног контекста и сходно томе пружа тачне одговоре.


Исправка верзије SQLite  
Ако наиђете на грешку:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Уклоните коментар са овог блока кода на почетку вашег нотебука:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Увоз пакета
Следећи код увози неопходне пакете:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Генерација уз помоћ претраживања са Семантичким Кернелом и Azure AI Agent Service

Овај пример показује како користити **Azure AI Agent Service** за извођење **генерације уз помоћ претраживања (RAG)** комбиновањем језичког модела са контекстом специфичним за одређену област из отпремљеног документа.

### Како функционише

1. **Отпремање документа**: Markdown датотека (document.md) која садржи информације (полиса путног осигурања компаније Contoso) се отпрема у агент сервис.

2. **Креирање векторске базе**: Документ се индексира у векторску базу како би се омогућила семантичка претрага његовог садржаја.

3. **Конфигурација агента**: Агент се инстанцира користећи модел `gpt-4o` са следећим строгим упутствима:
   - Одговара само на питања заснована на пронађеном садржају из документа.
   - Одбија да одговори ако је питање ван домена.

4. **Интеграција алата за претрагу датотека**: `FileSearchTool` се региструје код агента, омогућавајући моделу да претражује и преузима релевантне делове из индексираног документа током инференције.

5. **Интеракција са корисником**: Корисници могу постављати питања. Ако се релевантне информације пронађу у документу, агент генерише одговор заснован на тим информацијама.  
   Ако не, агент експлицитно одговара да документ не садржи довољно информација.


### Главна функција



Уверите се да прво покренете `az login` користећи Azure CLI како би био обезбеђен одговарајући контекст аутентификације приликом коришћења `DefaultAzureCredential`. Услуга Azure AI Agent не користи API кључеве.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Naravno! Molim vas da mi dostavite sadržaj markdown datoteke koju želite da prevedem.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
